In [ ]:
from src.load_wordbank_data import read_and_clean_wordbank_data
from src.wordbank_logistic_fits import compute_curve_fits, plot_curve_fits, compute_and_export_curve_fits, row_to_df_for_fit

In [ ]:
split_dict_WS = {'old_token':'inside/in',
              'new_tokens':['inside', 'in'],
              'new_lemmas':['inside', 'in']
              }
df_WS = read_and_clean_wordbank_data('~/Desktop/LingPredict data/wordbank_data_WS_Produces_en_US.csv',
                                        '~/Desktop/LingPredict data/uni_lemmas_WS_en_US.csv',
                                        lang='en', inventory='WS', measure='Produces',
                                        items_to_split=[split_dict_WS], cols_to_drop=['28'] # drop 28 month outlier
                                      )
df_WG = read_and_clean_wordbank_data('~/Desktop/LingPredict data/wordbank_data_WG_Produces_en_US.csv',
                                        '~/Desktop/LingPredict data/uni_lemmas_WG_en_US.csv',
                                        lang='en', inventory='WG', measure='Produces',
                                      )
df_WG_U = read_and_clean_wordbank_data('~/Desktop/LingPredict data/wordbank_data_WG_Understands_en_US.csv',
                                        '~/Desktop/LingPredict data/uni_lemmas_WG_en_US.csv',
                                        lang='en', inventory='WG', measure='Understands',
                                      )

In [ ]:
display(df_WS.head())
display(df_WS.tail())
df_WS.shape

In [ ]:
display(df_WG.head())
display(df_WG.tail())
df_WG.shape

In [ ]:
display(df_WG_U.head())
display(df_WG_U.tail())
df_WG_U.shape

In [ ]:
print(df_WG['uni_lemma'].isin(df_WS['uni_lemma']).all())

In [ ]:
df_WS[df_WS['synonym']==True]

In [ ]:
df_WG[df_WG['synonym']==True]

In [ ]:
df_WG[df_WG['token'].str.contains('/')]

In [ ]:
df_WS[df_WS['token'].str.contains('/')]

In [ ]:
print(df_WS[df_WS['token'].str.contains(r'\(')].shape)
df_WS[df_WS['token'].str.contains(r'\(')]

In [ ]:
print(df_WG[df_WG['token'].str.contains(r'\(')].shape)
df_WG[df_WG['token'].str.contains(r'\(')]

In [ ]:
# df_WS[df_WS['uni_lemma'].str.contains('\(')]

In [ ]:
# df_WG[df_WG['uni_lemma'].str.contains('\(')]

In [ ]:
# dfs = [df_WS, df_WG, df_WG_U]
# df_curve_fits = compute_curve_fits(dfs)

In [ ]:
for dat in [df_WG_U, df_WG, df_WS]:
    display(dat[dat['token']=='puppy'])

In [ ]:
display(df_WS[df_WS['token'].str.contains('call')])
df_WG[df_WG['token'].str.contains('call')]

In [ ]:
# plot_curve_fits(df_curve_fits)

In [ ]:
# print(df_curve_fits.sort_values(by='median_aoa', ascending=True)[['uni_lemma', 'growth_rate', 'median_aoa']].head(10))

In [ ]:
# print(df_curve_fits.drop(columns=['__plot_data__']).head())

In [ ]:
# compute_and_export_curve_fits('~/Desktop/wordbank_en_logistic_fits.csv', dfs)

In [ ]:
# df_curve_fits.sort_values(by='median_aoa', ascending=False)[['uni_lemma', 'token', 'growth_rate', 'median_aoa']].head(20)

In [ ]:
for dat in [df_WG_U, df_WG, df_WS]:
    display(dat[dat['uni_lemma']=='play pen'])

In [ ]:
for dat in [df_WG_U, df_WG, df_WS]:
    display(dat[dat['uni_lemma']=='babysitter'])

In [ ]:
dfs = [df_WS, df_WG]
df_curve_fits_produces, df_ambiguous = compute_curve_fits(dfs)

In [ ]:
df_ambiguous

In [ ]:
for index, row in df_ambiguous.iterrows():
    print(f"uni_lemma = {row['uni_lemma']}, token = {row['token']}")
    display(df_WS[df_WS['uni_lemma']==row['uni_lemma']])
    display(df_WG[df_WG['uni_lemma']==row['uni_lemma']])
    print(80*"=")

In [ ]:
dfs_U = [df_WG_U]
df_curve_fits_understands, df_ambiguous_understands = compute_curve_fits(dfs_U)

In [ ]:
plot_curve_fits(df_curve_fits_produces)

In [ ]:
plot_curve_fits(df_curve_fits_understands, colors=['#9e1cd6'])

In [ ]:
print(df_curve_fits_produces.head())
df_curve_fits_produces.shape

In [ ]:
display(df_curve_fits_understands.head())
df_curve_fits_understands.shape

In [ ]:
#compute_and_export_curve_fits('~/Desktop/wordbank_en_logistic_fits.csv', [df_WS, df_WG], [df_WG_U])

In [ ]:
# import pandas as pd

# fits = pd.read_csv('~/Desktop/wordbank_en_logistic_fits.csv')
# fits.head(25)

In [ ]:
df_WS['category'].unique()

In [ ]:
import pandas as pd

def plot_word_side_by_side(df_data_all: pd.DataFrame, word):
    """
    Generates the side-by-side plot comparison for a give word.
    Left: WG Comprehension only.
    Right: WG Production + WS Production combined.
    """
    # 1. Define the two scenarios
    # Scenario 1 (Left Plot): WG Understands (Comprehension)
    df_left = df_data_all[
        (df_data_all['inventory'] == 'WG') & 
        (df_data_all['measure'] == 'comprehension')
    ]
    
    # Scenario 2 (Right Plot): WG and WS Produces (Production)
    df_right = df_data_all[
        (df_data_all['measure'] == 'production')
    ]

    # Check if we have data for both
    if df_left.empty or df_right.empty:
        print("Error: Could not find required data subsets for both comparison plots.")
        return

    # 2. Compute the fits for each scenario
    # We use a list containing only the required DataFrame for 'compute_curve_fits'
    # The 'compute_curve_fits' will return the fitted data in its first return value
    df_fits_left, _ = compute_curve_fits([df_left])
    df_fits_right, _ = compute_curve_fits([df_right])

    # Since we are filtering for a single word and single fit result (one row each), 
    # we extract the first (and only) row's data
    row_left = df_fits_left.iloc[0]
    row_right = df_fits_right.iloc[0]

    # 3. Create the figure and axes
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    
    # 4. Plot the Left side (WG Comprehension)
    plot_acquisition_curve(
        ax=axes[0],
        word='bus (WG Comprehension)',
        df_data=row_left['__plot_data__'],
        k_fit=row_left['growth_rate'],
        x0_fit=row_left['median_aoa'],
        colors=['#0000FF'] # Use one distinct color
    )
    axes[0].legend(loc='lower right', fontsize=6)
    
    # 5. Plot the Right side (WG/WS Production)
    plot_acquisition_curve(
        ax=axes[1],
        word='bus (WG & WS Production)',
        df_data=row_right['__plot_data__'],
        k_fit=row_right['growth_rate'],
        x0_fit=row_right['median_aoa'],
        colors=['#FFA500', '#9e1cd6'] # Use two distinct colors for the two measures
    )
    # Add a single legend for the combined plot (since plot_acquisition_curve removes it)
    # Re-plot the lines on the right axis to get the handles for the legend
    lines, labels = axes[1].get_legend_handles_labels()
    
    # Manually collect scatter handles from the right plot data
    # Create the legend labels based on the data in row_right['__plot_data__']
    legend_elements = []
    
    # Create manual handles for the scatter points for the right plot
    labels_right = row_right['__plot_data__']['_inv_measure'].unique()
    custom_colors = ['#FFA500', '#9e1cd6']
    for i, label in enumerate(labels_right):
        legend_elements.append(
            plt.Line2D([0], [0], marker='o', color='w', label=label, 
                       markerfacecolor=custom_colors[i % len(custom_colors)], 
                       markeredgecolor='black', markersize=5)
        )
    
    # Add fitted line elements
    legend_elements.extend([lines[0], lines[1], lines[2]])

    axes[1].legend(handles=legend_elements, loc='lower right', fontsize=6)


    # 6. Final adjustments and display
    fig.suptitle("Acquisition Curve Comparison for 'bus'", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
bus = []
for dat in [df_WS, df_WG, df_WG_U]:
    bus.append(dat[dat['token_clean']=='bus'])

In [ ]:
row_to_df_for_fit(bus[0])

In [ ]:
plot_curve_fits(df_curve_fits_produces[df_curve_fits_produces['token_clean']=='bus'], cols=1)

In [ ]:
plot_curve_fits(df_curve_fits_understands[df_curve_fits_understands['token_clean']=='bus'], cols=1, colors=['#9e1cd6'])